# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [ ]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [ ]:
# Your code here:
# Check the dimensions of the dataset
rows, cols = df.shape
print(f"Our inventory has {rows:,} diamonds and {cols} attributes.")

Our inventory has 6,000 diamonds and 8 attributes.


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [ ]:
# Your code here:
print("We track the following information about each diamond:")
print(df.columns)

We track the following information about each diamond:
['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms: Each row is a unique diamond and information regarding characteristics of it.

**Your answer:** Each row represents: a unique diamond in the database

## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [6]:
# Your code here:
total_carats = df.select(pl.sum("Carat Weight")).item()
print(f"The total carats in our inventory is {total_carats:.2f} carats.")

The total carats in our inventory is 8007.12 carats.


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [8]:
# Your code here:
unique_cuts = df.select(pl.col("Cut").unique()).to_series().to_list()
num_cuts = len(unique_cuts)

print(f"We track {num_cuts} unique cuts in our inventory:")
for cut in unique_cuts:
    print(f"- {cut}")

We track 5 unique cuts in our inventory:
- Fair
- Ideal
- Signature-Ideal
- Good
- Very Good


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [10]:
# Your code here:
most_expensive = df.select(pl.max("Price")).item()
least_expensive = df.select(pl.min("Price")).item()

print(f"Our most valuable diamond is priced at ${most_expensive:,}.")
print(f"Our least valuable diamond is priced at ${least_expensive:,}.")

Our most valuable diamond is priced at $101,561.
Our least valuable diamond is priced at $2,184.


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [12]:
# Your code here:
avg_price_by_color = (
    df.group_by("Color")
      .agg(pl.mean("Price").alias("avg_price"))
      .sort("avg_price", descending=True)
)

print(avg_price_by_color)

shape: (6, 2)
┌───────┬──────────────┐
│ Color ┆ avg_price    │
│ ---   ┆ ---          │
│ str   ┆ f64          │
╞═══════╪══════════════╡
│ D     ┆ 15255.783661 │
│ F     ┆ 12712.241856 │
│ G     ┆ 12520.050633 │
│ E     ┆ 11539.190231 │
│ H     ┆ 10487.347544 │
│ I     ┆ 8989.636364  │
└───────┴──────────────┘


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:** New grain = a color category, and summarizes all diamonds of that color

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [14]:
clarity_summary = (
    df.group_by("Clarity")
      .agg([
          pl.count().alias("diamond_count"),
          pl.col("Price").mean().round(2).alias("avg_price"),
          pl.col("Carat Weight").sum().round(2).alias("total_carats")
      ])
      .sort("avg_price", descending=True)  # optional, sorts by typical price
)

print(clarity_summary)

shape: (7, 4)
┌─────────┬───────────────┬───────────┬──────────────┐
│ Clarity ┆ diamond_count ┆ avg_price ┆ total_carats │
│ ---     ┆ ---           ┆ ---       ┆ ---          │
│ str     ┆ u32           ┆ f64       ┆ f64          │
╞═════════╪═══════════════╪═══════════╪══════════════╡
│ FL      ┆ 4             ┆ 63776.0   ┆ 7.87         │
│ IF      ┆ 219           ┆ 22105.84  ┆ 316.0        │
│ VVS1    ┆ 285           ┆ 16845.68  ┆ 389.43       │
│ VVS2    ┆ 666           ┆ 14142.18  ┆ 876.58       │
│ VS1     ┆ 1192          ┆ 13694.11  ┆ 1682.74      │
│ VS2     ┆ 1575          ┆ 11809.05  ┆ 2170.81      │
│ SI1     ┆ 2059          ┆ 8018.86   ┆ 2563.69      │
└─────────┴───────────────┴───────────┴──────────────┘


/tmp/ipython-input-802250209.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("diamond_count"),


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [18]:
# Your code here:
cut_color_summary = (
    df.group_by(["Cut", "Color"])
      .agg([
          pl.col("Price").mean().round(2).alias("avg_price"),
          pl.len().alias("diamond_count")  # optional: shows how many in each group
      ])
      .sort(["Cut", "Color"])
)

print(cut_color_summary)

shape: (30, 4)
┌───────────┬───────┬───────────┬───────────────┐
│ Cut       ┆ Color ┆ avg_price ┆ diamond_count │
│ ---       ┆ ---   ┆ ---       ┆ ---           │
│ str       ┆ str   ┆ f64       ┆ u32           │
╞═══════════╪═══════╪═══════════╪═══════════════╡
│ Fair      ┆ D     ┆ 6058.25   ┆ 12            │
│ Fair      ┆ E     ┆ 5370.63   ┆ 32            │
│ Fair      ┆ F     ┆ 6063.63   ┆ 24            │
│ Fair      ┆ G     ┆ 7345.52   ┆ 21            │
│ Fair      ┆ H     ┆ 5908.5    ┆ 24            │
│ …         ┆ …     ┆ …         ┆ …             │
│ Very Good ┆ E     ┆ 12101.91  ┆ 323           │
│ Very Good ┆ F     ┆ 12413.91  ┆ 455           │
│ Very Good ┆ G     ┆ 12354.01  ┆ 578           │
│ Very Good ┆ H     ┆ 10056.11  ┆ 424           │
│ Very Good ┆ I     ┆ 8930.03   ┆ 383           │
└───────────┴───────┴───────────┴───────────────┘


**Reflection:** How specific is your analysis now? What level of detail does each row represent? - the analysis is now more granular.

**Your answer:** New grain = diamonds that share the same cut and color. One specific batch of diamonds.

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [20]:
# Your code here:
inventory_segments = (
    df.group_by(["Cut", "Color"])
      .agg([
          pl.sum("Price").alias("total_value"),    # total inventory value
          pl.len().alias("diamond_count")        # number of diamonds in this segment
      ])
      .sort("total_value", descending=True)       # top segments first
)

# Show top 5 most valuable segments
top_5_segments = inventory_segments.head(5)
print(top_5_segments)

shape: (5, 4)
┌───────────┬───────┬─────────────┬───────────────┐
│ Cut       ┆ Color ┆ total_value ┆ diamond_count │
│ ---       ┆ ---   ┆ ---         ┆ ---           │
│ str       ┆ str   ┆ i64         ┆ u32           │
╞═══════════╪═══════╪═════════════╪═══════════════╡
│ Ideal     ┆ G     ┆ 9363514     ┆ 690           │
│ Very Good ┆ G     ┆ 7140620     ┆ 578           │
│ Very Good ┆ F     ┆ 5648327     ┆ 455           │
│ Ideal     ┆ F     ┆ 5346782     ┆ 363           │
│ Ideal     ┆ H     ┆ 5279687     ┆ 458           │
└───────────┴───────┴─────────────┴───────────────┘


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use

In [21]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

# Your code here:
rows, cols = df.shape
print(f"Our inventory has {rows:,} diamonds and {cols} attributes.")

Loading diamonds dataset...
Dataset loaded successfully!
Our inventory has 6,000 diamonds and 8 attributes.
